## Import and control data

In [30]:
# prove path is correct
import os
BIDS_ROOT = r"C:\\Users\\Om\\Downloads\\ds000030\\"
print(os.path.exists(BIDS_ROOT))
print(os.listdir(BIDS_ROOT)[:10])

import sys
print("Notebook Python:", sys.executable)
import subprocess
import glob

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, r2_score

import matplotlib.pyplot as plt

import dipy, nilearn
import nibabel as nib

# imaging libraries
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
from dipy.segment.mask import median_otsu

True
['dataset_description.json', 'derivatives', 'ds000030_imaging_summary.csv', 'participants.tsv', 'phenotype', 'sub-10159', 'sub-10171', 'sub-10189', 'sub-10193', 'sub-10206']
Notebook Python: c:\Program Files\Python312\python.exe


In [ ]:
# %% 
# ----- PATHS -----
BIDS_ROOT = r"C:\Users\Om\Downloads\ds000030"  # folder with subset
PHENO_DIR = os.path.join(BIDS_ROOT, "phenotype")   
OUTPUT_DIR = os.path.join(BIDS_ROOT, "derivatives", "project_outputs")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# list subjects present in the subset
subjects = sorted(
    [
        d for d in os.listdir(BIDS_ROOT)
        if d.startswith("sub-") and os.path.isdir(os.path.join(BIDS_ROOT, d))
    ]
)

print(f"There are {len(subjects)} subjects in subset.")
print(subjects[:10])


There are 30 subjects in subset.
['sub-10159', 'sub-10171', 'sub-10189', 'sub-10193', 'sub-10206', 'sub-10217', 'sub-10225', 'sub-10227', 'sub-10228', 'sub-10235']


In [32]:
# load participants file

participants_path = os.path.join(BIDS_ROOT, "participants.tsv")
participants = pd.read_csv(participants_path, sep="\t")

print(participants.shape)
participants.head()


(272, 16)


,participant_id,diagnosis,age,gender,bart,bht,dwi,pamenc,pamret,rest,scap,stopsignal,T1w,taskswitch,ScannerSerialNumber,ghost_NoGhost
0,sub-10159,CONTROL,30,F,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
1,sub-10171,CONTROL,24,M,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
2,sub-10189,CONTROL,49,M,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
3,sub-10193,CONTROL,40,M,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,35343.0,No_ghost
4,sub-10206,CONTROL,21,M,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost


In [33]:
participants.columns

Index(['participant_id', 'diagnosis', 'age', 'gender', 'bart', 'bht', 'dwi',
       'pamenc', 'pamret', 'rest', 'scap', 'stopsignal', 'T1w', 'taskswitch',
       'ScannerSerialNumber', 'ghost_NoGhost'],
      dtype='object')

In [34]:
# load phenotype data

pheno_file = os.path.join(PHENO_DIR, "ADHD.tsv")

if os.path.exists(pheno_file):
    pheno = pd.read_csv(pheno_file, sep="\t")
    print("Phenotype shape:", pheno.shape)
    pheno.head()
else:
    print("Phenotype file not found at:", pheno_file)
    pheno = pd.DataFrame()

Phenotype shape: (272, 12)


In [35]:
# check for DWI and T1w files for each subject
import glob

subjects = sorted([d for d in os.listdir(BIDS_ROOT) if d.startswith("sub-")])

results = []

for sub in subjects:
    dwi_path = glob.glob(os.path.join(BIDS_ROOT, sub, "dwi", "*dwi.nii.gz"))
    t1_path = glob.glob(os.path.join(BIDS_ROOT, sub, "anat", "*T1w.nii.gz"))

    results.append({
        "subject": sub,
        "has_dwi": bool(dwi_path),
        "has_t1": bool(t1_path)
    })

check_df = pd.DataFrame(results)
check_df


,subject,has_dwi,has_t1
0,sub-10159,True,True
1,sub-10171,True,True
2,sub-10189,True,True
3,sub-10193,True,True
4,sub-10206,True,True
5,sub-10217,True,True
6,sub-10225,True,True
7,sub-10227,True,True
8,sub-10228,True,True
9,sub-10235,True,True


In [36]:
# run the DTI extraction pipeline

dti_rows = []

for sub in subjects:
    dwi_dir = os.path.join(BIDS_ROOT, sub, "dwi")
    if not os.path.isdir(dwi_dir):
        continue

    dwi_file = glob.glob(os.path.join(dwi_dir, "*dwi.nii.gz"))
    bval_file = glob.glob(os.path.join(dwi_dir, "*.bval"))
    bvec_file = glob.glob(os.path.join(dwi_dir, "*.bvec"))

    if not dwi_file or not bval_file or not bvec_file:
        continue

    dwi_file = dwi_file[0]
    bval_file = bval_file[0]
    bvec_file = bvec_file[0]

    print(f"Processing DTI for: {sub}")

    # load DWI data
    img = nib.load(dwi_file)
    data = img.get_fdata()

    # load bvals & bvecs
    bvals, bvecs = read_bvals_bvecs(bval_file, bvec_file)
    gtab = gradient_table(bvals, bvecs)

    # make mask
    data_masked, mask = median_otsu(data, vol_idx=range(10), median_radius=3)

    # fit tensor
    tenmodel = TensorModel(gtab)
    tenfit = tenmodel.fit(data_masked)

    FA = tenfit.fa[mask]
    MD = tenfit.md[mask]

    dti_rows.append({
        "participant_id": sub,
        "FA_mean": np.nanmean(FA),
        "FA_std": np.nanstd(FA),
        "MD_mean": np.nanmean(MD),
        "MD_std": np.nanstd(MD),
    })

dti_df = pd.DataFrame(dti_rows)
print("Finished DTI extraction for", len(dti_df), "subjects.", "Shape:", dti_df.shape, ".")
dti_df.head()

Processing DTI for: sub-10159


C:\Users\Om\AppData\Local\Temp\ipykernel_33564\1846746705.py:29: UserWarning: Pass ['bvecs'] as keyword args. From version 2.0.0 passing these as positional arguments will result in an error. 
  gtab = gradient_table(bvals, bvecs)


Processing DTI for: sub-10171
Processing DTI for: sub-10189
Processing DTI for: sub-10193
Processing DTI for: sub-10206
Processing DTI for: sub-10217
Processing DTI for: sub-10225
Processing DTI for: sub-10227
Processing DTI for: sub-10228
Processing DTI for: sub-10235
Processing DTI for: sub-10249
Processing DTI for: sub-10269
Processing DTI for: sub-10271
Processing DTI for: sub-10273
Processing DTI for: sub-10274
Processing DTI for: sub-10280
Processing DTI for: sub-10290
Processing DTI for: sub-10292
Processing DTI for: sub-10304
Processing DTI for: sub-10316
Processing DTI for: sub-10321
Processing DTI for: sub-10325
Processing DTI for: sub-10329
Processing DTI for: sub-10339
Processing DTI for: sub-10340
Processing DTI for: sub-10345
Processing DTI for: sub-10347
Processing DTI for: sub-10356
Processing DTI for: sub-10361
Finished DTI extraction for 29 subjects. Shape: (29, 5) .


,participant_id,FA_mean,FA_std,MD_mean,MD_std
0,sub-10159,0.276670,0.187389,0.001140,0.017854
1,sub-10171,0.284232,0.180213,0.002911,0.598804
2,sub-10189,0.300255,0.209072,0.001373,0.101038
3,sub-10193,0.295395,0.187221,0.001944,0.374821
4,sub-10206,0.308312,0.207156,0.004249,0.543643


In [37]:
# run the T1 structural summary extraction

from nilearn.image import load_img

t1_rows = []

for sub in subjects:
    anat_dir = os.path.join(BIDS_ROOT, sub, "anat")
    if not os.path.isdir(anat_dir):
        continue

    t1_files = glob.glob(os.path.join(anat_dir, "*T1w.nii.gz"))
    if not t1_files:
        continue

    t1_file = t1_files[0]
    print(f"Processing T1 for: {sub}")

    # load T1 image
    img = load_img(t1_file)
    data = img.get_fdata()

    # compute brain mask vota thresholding
    thresh = np.percentile(data, 60)
    brain_mask = data > thresh

    brain = data[brain_mask]

    t1_rows.append({
        "participant_id": sub,
        "T1_mean_intensity": np.nanmean(brain),
        "T1_intensity_std": np.nanstd(brain),
        "T1_brain_voxels": brain_mask.sum(),
    })

t1_df = pd.DataFrame(t1_rows)
print("Finished T1 extraction for:", len(t1_df), ". Shape:", t1_df.shape)
t1_df.head()

Processing T1 for: sub-10159
Processing T1 for: sub-10171
Processing T1 for: sub-10189
Processing T1 for: sub-10193
Processing T1 for: sub-10206
Processing T1 for: sub-10217
Processing T1 for: sub-10225
Processing T1 for: sub-10227
Processing T1 for: sub-10228
Processing T1 for: sub-10235
Processing T1 for: sub-10249
Processing T1 for: sub-10269
Processing T1 for: sub-10271
Processing T1 for: sub-10273
Processing T1 for: sub-10274
Processing T1 for: sub-10280
Processing T1 for: sub-10290
Processing T1 for: sub-10292
Processing T1 for: sub-10304
Processing T1 for: sub-10316
Processing T1 for: sub-10321
Processing T1 for: sub-10325
Processing T1 for: sub-10329
Processing T1 for: sub-10339
Processing T1 for: sub-10340
Processing T1 for: sub-10345
Processing T1 for: sub-10347
Processing T1 for: sub-10356
Processing T1 for: sub-10361
Finished T1 extraction for: 29 . Shape: (29, 4)


,participant_id,T1_mean_intensity,T1_intensity_std,T1_brain_voxels
0,sub-10159,128.524192,109.631080,4454488
1,sub-10171,157.747413,112.704454,4482037
2,sub-10189,157.799750,116.845960,4555075
3,sub-10193,157.097853,113.923768,4521360
4,sub-10206,130.596633,97.857776,4394794


In [38]:
# combine DTI and T1 summaries into one dataframe

imaging_df = pd.merge(dti_df, t1_df, on="participant_id", how="outer")
print(imaging_df.shape)
imaging_df.head()

out_path = os.path.join(BIDS_ROOT, "ds000030_imaging_summary.csv")
imaging_df.to_csv(out_path, index=False)
print("Saved:", out_path)

(29, 8)
Saved: C:\Users\Om\Downloads\ds000030\ds000030_imaging_summary.csv


## Statistical Analysis!

In [42]:
# statistical analysis

imaging_file = os.path.join(BIDS_ROOT, "ds000030_imaging_summary.csv")

IMAGING_COLS = [
    "FA_mean", "FA_std",
    "MD_mean", "MD_std",
    "T1_mean_intensity",
    "T1_intensity_std",
    "T1_brain_voxels",
]

# load imaging summary file

imaging_df = pd.read_csv(imaging_file)
imaging_df.head()

,participant_id,FA_mean,FA_std,MD_mean,MD_std,T1_mean_intensity,T1_intensity_std,T1_brain_voxels
0,sub-10159,0.276670,0.187389,0.001140,0.017854,128.524192,109.631080,4454488
1,sub-10171,0.284232,0.180213,0.002911,0.598804,157.747413,112.704454,4482037
2,sub-10189,0.300255,0.209072,0.001373,0.101038,157.799750,116.845960,4555075
3,sub-10193,0.295395,0.187221,0.001944,0.374821,157.097853,113.923768,4521360
4,sub-10206,0.308312,0.207156,0.004249,0.543643,130.596633,97.857776,4394794


In [ ]:
# load behavioral/phenotypic dataset
# !!!!!!THIS NEEDS DEFINITIVE COLUMNS THAT YOU NEED TO CHOOSE!!!!!!

pheno = pd.read_csv(os.path.join(BIDS_ROOT, "phenotype", "ADHD.tsv"), sep="\t")
pheno.columns

Index(['participant_id', 'adhd5', 'adhd4', 'adhd7', 'adhd2', 'adhd8', 'adhd6',
       'adhd1', 'adhd10', 'adhd11', 'adhd9', 'adhd3'],
      dtype='object')

In [47]:
# merge particpants, imaging, and phenotype data into one dataframe

master = participants.merge(pheno, on="participant_id", how="left")
master = master.merge(imaging_df, on="participant_id", how="left")

print(master.shape)
master.head()

(272, 34)


,participant_id,diagnosis,age,gender,bart,bht,dwi,pamenc,pamret,rest,...,adhd11,adhd9,adhd3,FA_mean,FA_std,MD_mean,MD_std,T1_mean_intensity,T1_intensity_std,T1_brain_voxels
0,sub-10159,CONTROL,30,F,1.0,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,0.276670,0.187389,0.001140,0.017854,128.524192,109.631080,4454488.0
1,sub-10171,CONTROL,24,M,1.0,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,0.284232,0.180213,0.002911,0.598804,157.747413,112.704454,4482037.0
2,sub-10189,CONTROL,49,M,1.0,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,0.300255,0.209072,0.001373,0.101038,157.799750,116.845960,4555075.0
3,sub-10193,CONTROL,40,M,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.295395,0.187221,0.001944,0.374821,157.097853,113.923768,4521360.0
4,sub-10206,CONTROL,21,M,1.0,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,0.308312,0.207156,0.004249,0.543643,130.596633,97.857776,4394794.0
